In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed

2025-06-19 18:41:51.570815: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-19 18:41:54.324021: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Input shape: (batch_size, 13, 32)
input_seq = Input(shape=(13, 32), name='input_sequence')

# Encoder LSTM
encoder_output, state_h, state_c = LSTM(64, return_state=True, name='encoder_lstm')(input_seq)
encoder_states = [state_h, state_c]

# Repeat context vector for each of the 7 future time points
decoder_input = RepeatVector(7)(encoder_output)

# Decoder LSTM
decoder_lstm = LSTM(64, return_sequences=True, name='decoder_lstm')
decoder_output = decoder_lstm(decoder_input, initial_state=encoder_states)

# Output dense layer to produce 1 value per time step
output = TimeDistributed(Dense(1), name='output')(decoder_output)

# Define and compile model
model = Model(inputs=input_seq, outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.summary()

2025-06-19 18:42:51.609720: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :
2025-06-19 18:42:51.609781: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2025-06-19 18:42:51.609813: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c2522.swan.hcc.unl.edu): /proc/driver/nvidia/version does not exist
2025-06-19 18:42:51.610214: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_sequence (InputLayer)    [(None, 13, 32)]     0           []                               
                                                                                                  
 encoder_lstm (LSTM)            [(None, 64),         24832       ['input_sequence[0][0]']         
                                 (None, 64),                                                      
                                 (None, 64)]                                                      
                                                                                                  
 repeat_vector (RepeatVector)   (None, 7, 64)        0           ['encoder_lstm[0][0]']           
                                                                                              